# Pandas Overview

The purpose of this notebook is to offer an overview of pandas features via a concrete data analysis
Dataset details : https://www.kaggle.com/datasets/murderaccountability/homicide-reports

### Dependencies

In [1]:
%matplotlib inline
from IPython.display import display
import numpy as np
import pandas as pd 
import re
import matplotlib.pylab as plt
import seaborn as sns

### Pandas configuration

In [2]:
#misc
plt.style.use('ggplot')

# pandas
pd.options.display.min_rows = 20
pd.options.display.max_columns = 200
pd.options.display.max_colwidth = 25
pd.options.display.expand_frame_repr = 100
pd.options.display.max_rows = 50 # None = no limitation
pd.options.display.max_seq_items = 50 # Max length of printed sequence
pd.options.display.precision = 4
pd.options.display.show_dimensions = False
pd.__version__

'1.5.1'

### Helpers

In [3]:
def clean_title(title):
    title = re.sub('[-_ ]', ' ', title)
    return ''.join(word.capitalize() for word in title.split(' '))

In [4]:
def millify(num):
    return f"{num:,}"

In [5]:
def get_var(df, var_name):
    globals()[var_name] = df
    return df

### Data import

In [6]:
# read raw data
crimes = pd.read_csv(
    'homicide_reports_1980-2014.zip',
    low_memory=False
) 
millify(crimes.memory_usage().sum())

'122,583,296'

### Data understanding

In [7]:
crimes.shape

(638454, 24)

In [8]:
# display 5 lines : head(5), tail(5), sample(5)
crimes.sample(5)

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,Crime Solved,Victim Sex,Victim Age,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
486182,486183,FL00612,Davie,Municipal Police,Broward,Florida,2005,March,768,Murder or Manslaughter,Yes,Male,1,White,Unknown,Male,24,White,Unknown,Son,Unknown,0,0,FOIA
610070,610071,WI06500,Walworth County,Sheriff,Walworth,Wisconsin,2013,October,1,Manslaughter by Negli...,Yes,Female,40,White,Unknown,Male,50,White,Unknown,Stranger,Unknown,0,0,FBI
625440,625441,CA01942,Los Angeles,Municipal Police,Los Angeles,California,2014,November,19,Murder or Manslaughter,Yes,Male,29,White,Hispanic,Male,41,White,Hispanic,Unknown,Blunt Object,0,1,FBI
451992,451993,CA03404,Sacramento,Municipal Police,Sacramento,California,2003,December,5,Murder or Manslaughter,No,Male,43,Asian/Pacific Islander,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0,FBI
545927,545928,PAPPD00,Pittsburgh,Municipal Police,Allegheny,Pennsylvania,2008,October,1,Murder or Manslaughter,No,Male,43,Black,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,0,FBI


In [9]:
crimes.dtypes

Record ID                 int64
Agency Code              object
Agency Name              object
Agency Type              object
City                     object
State                    object
Year                      int64
Month                    object
Incident                  int64
Crime Type               object
Crime Solved             object
Victim Sex               object
Victim Age                int64
Victim Race              object
Victim Ethnicity         object
Perpetrator Sex          object
Perpetrator Age          object
Perpetrator Race         object
Perpetrator Ethnicity    object
Relationship             object
Weapon                   object
Victim Count              int64
Perpetrator Count         int64
Record Source            object
dtype: object

In [10]:
crimes.columns

Index(['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City',
       'State', 'Year', 'Month', 'Incident', 'Crime Type', 'Crime Solved',
       'Victim Sex', 'Victim Age', 'Victim Race', 'Victim Ethnicity',
       'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race',
       'Perpetrator Ethnicity', 'Relationship', 'Weapon', 'Victim Count',
       'Perpetrator Count', 'Record Source'],
      dtype='object')

In [11]:
crimes.index

RangeIndex(start=0, stop=638454, step=1)

In [12]:
crimes.describe()

,Record ID,Year,Incident,Victim Age,Victim Count,Perpetrator Count
count,638454.0000,638454.0000,638454.0000,638454.0000,638454.0000,638454.0000
mean,319227.5000,1995.8011,22.9679,35.0335,0.1233,0.1852
std,184305.9387,9.9277,92.1498,41.6283,0.5377,0.5855
min,1.0000,1980.0000,0.0000,0.0000,0.0000,0.0000
25%,159614.2500,1987.0000,1.0000,22.0000,0.0000,0.0000
50%,319227.5000,1995.0000,2.0000,30.0000,0.0000,0.0000
75%,478840.7500,2004.0000,10.0000,42.0000,0.0000,0.0000
max,638454.0000,2014.0000,999.0000,998.0000,10.0000,10.0000


### Data tweaking

In [13]:
# tweek data
def tweak_crimes(df_):

    return (df_
        .rename(columns=clean_title)
        .assign(
            PerpetratorAge=lambda df_ : pd.to_numeric(df_['PerpetratorAge'], errors ='coerce').fillna(0),
            CrimeDate=pd.to_datetime(df_['Year'].astype(str) + ' ' + df_['Month'].astype(str) + ' 1', format="%Y %B %d")                
        )
        .astype({
            'CrimeSolved' : 'bool',
            **{ name : 'category' for name in [
                'AgencyCode', 
                'AgencyName', 
                'AgencyType',
                'City', 
                'State', 
                'CrimeType',
                'VictimSex',
                'VictimRace',
                'VictimEthnicity',
                'PerpetratorSex',
                'PerpetratorRace',
                'PerpetratorEthnicity',
                'Relationship',
                'Weapon',
                'RecordSource'                            
            ]},
            **{ name : np.uint8 for name in [
                'Incident',
                'VictimAge',
                'VictimCount',
                'PerpetratorAge',
                'PerpetratorCount'
            ]}
        })
        .set_index("RecordId")
        .drop(columns=['Month', 'Year'])     
    #     .pipe(lambda df: display(df) or df )
    #     .pipe(get_var, 'temp')
)

crimes = tweak_crimes(crimes)

### Data display

In [14]:
# memory consumption
millify(crimes.memory_usage(deep=True).sum())

'27,669,047'

In [15]:
# types and memory consumption
crimes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 638454 entries, 1 to 638454
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   AgencyCode            638454 non-null  category      
 1   AgencyName            638454 non-null  category      
 2   AgencyType            638454 non-null  category      
 3   City                  638454 non-null  category      
 4   State                 638454 non-null  category      
 5   Incident              638454 non-null  uint8         
 6   CrimeType             638454 non-null  category      
 7   CrimeSolved           638454 non-null  bool          
 8   VictimSex             638454 non-null  category      
 9   VictimAge             638454 non-null  uint8         
 10  VictimRace            638454 non-null  category      
 11  VictimEthnicity       638454 non-null  category      
 12  PerpetratorSex        638454 non-null  category      
 13 

### Data aggregation

In [25]:
minVictims=0

(crimes
 
     # filtering (could use query, where, loc, pipe, mask)
    .query('VictimSex != "Unknown"')    
#     .where(lambda x: x['VictimSex'].ne('Unknown'))
#     .loc[lambda x: x['VictimSex'].ne('Unknown')]
#     .pipe(lambda x: x[x['VictimSex'].ne('Unknown')]) 
#     .mask(lambda x: x['VictimSex'].eq('Unknown')) 
 
    
     # grouping data by year, ...
    .groupby([
        pd.Grouper(key="CrimeDate", freq="Y"), 
        'State',
        'VictimSex' 
    ] )
 
    # custom aggregations
    .agg(         
         TotalVictims = ('VictimCount', 'sum'),
         AvgVictimAge = ('VictimAge', lambda serie : serie.mean()),
         AvgPerpetratorAge = ('PerpetratorAge', 'mean'),
    )

     # filtering
     .query('TotalVictims > @minVictims')
     .dropna()
 
     # unpivot data, ie move to columns (could use melt)
     .unstack('VictimSex') 
            
)

TotalVictims        AvgVictimAge           \
VictimSex                       Female   Male       Female     Male   
CrimeDate  State                                                      
1980-12-31 Alabama                17.0   25.0      38.7252  36.8455   
           Alaska                  4.0    2.0      35.8571  29.3030   
           Arizona                 9.0   13.0      39.1842  33.5324   
           Arkansas                9.0    9.0      29.5484  35.4605   
           California            121.0  201.0      34.5285  33.5100   
           Colorado                3.0   15.0      37.0000  33.6838   
           Connecticut             2.0    4.0      37.3143  35.4553   
           Delaware                4.0    4.0      29.4545  32.3939   
           Florida                44.0  104.0      36.3051  36.5209   
           Georgia                10.0   14.0      35.0367  36.5574   
...                                ...    ...          ...      ...   
2014-12-31 South Carolina         23.0   47.0      45.1282  33.5814   
           South Dakota            4.0    2.0      44.1111  35.5455   
           Tennessee               7.0   35.0      35.3733  33.8702   
           Texas                  86.0  110.0      37.8506  34.6084   
           Utah                   44.0   14.0      30.6667  30.2564   
           Vermont                 NaN    4.0          NaN  35.1250   
           Virginia               39.0   21.0      42.1250  34.4514   
           Washington             20.0   15.0      31.8421  36.6190   
           West Virginia           6.0    8.0      37.4783  50.6471   
           Wisconsin               6.0   10.0      33.0227  33.2126   

                          AvgPerpetratorAge           
VictimSex                            Female     Male  
CrimeDate  State                                      
1980-12-31 Alabama                  30.8397  30.3822  
           Alaska                   24.6429  26.2727  
           Arizona                  25.0263  22.4213  
           Arkansas                 24.8065  31.2632  
           California               21.0385  17.6040  
           Colorado                 19.0000  19.9412  
           Connecticut              24.8286  22.0650  
           Delaware                 23.3636  21.6970  
           Florida                  21.8550  21.0606  
           Georgia                  24.6239  25.3634  
...                                     ...      ...  
2014-12-31 South Carolina           35.2949  23.7907  
           South Dakota             41.7778  29.5455  
           Tennessee                34.4267  22.2596  
           Texas                    31.6220  22.0622  
           Utah                     37.8182  28.9487  
           Vermont                      NaN  51.3750  
           Virginia                 35.8125  23.7588  
           Washington               32.8947  29.9320  
           West Virginia            39.9565  32.8235  
           Wisconsin                33.4545  23.0236

In [162]:
(crimes
    .groupby(
        pd.Grouper(key="CrimeDate", freq="M"), 
    )
    .agg(
        VictimsSum = ('VictimCount', 'sum'),
    ) 
    .assign(
        VictimsChange = lambda x : x['VictimsSum'].astype(int).pct_change(),
        VictimsDiff = lambda x : x['VictimsSum'].astype(float).diff()        
    )
    
    .style
        .format({'VictimsChange' : '{:.2f}'}, na_rep='N/A',precision=0)
        .bar(subset=['VictimsChange'], align=0, cmap="bwr", height=50)
        .background_gradient(subset=['VictimsSum'], cmap='Reds')
)

,VictimsSum,VictimsChange,VictimsDiff
CrimeDate,,,
1980-01-31 00:00:00,228,N/A,N/A
1980-02-29 00:00:00,164,-0.28,-64
1980-03-31 00:00:00,160,-0.02,-4
1980-04-30 00:00:00,194,0.21,34
1980-05-31 00:00:00,208,0.07,14
1980-06-30 00:00:00,168,-0.19,-40
1980-07-31 00:00:00,208,0.24,40
1980-08-31 00:00:00,178,-0.14,-30
1980-09-30 00:00:00,190,0.07,12
